# GHZ Game
In this code we simulate the thought experiment known as the GHZ (Greenberger, Horne, Zeilinger) game

We have three parties: "Alice", "Bob" and "Charlie". Each one receives, respectively, bits x, y and z from $\{000, 011, 101, 110\}$ as inputs, then each one responds with bits a, b and c. They win the game if a, b and c are such that the following is satisfied
$$a \oplus b \oplus c = x \vee y \vee z$$
They can talk and decide a strategy beforehand, but they are unable to comunicate during the game. Classicaly, the best strategy guarantees that they will win with probability $0.75$. But if instead of outputing classical bits they output the measurements of the shared quantum state
$$|\psi\rangle = \frac{|000\rangle - |011\rangle - |101\rangle - |110\rangle}{2}$$
it can be shown that they can always win.

It is possible to use this experiment to chech whether the device we are running the code is actually quantum.

The strategy is as follows:

- If a player receives a $0$ he/she measures his/her qubit and outputs the result, if he/she receives a $1$ then a Hadamard must be applied before.

In [1]:
%matplotlib inline

from qiskit import *
from qiskit.tools.monitor import *
from qiskit.visualization import *
from qiskit.providers.ibmq import least_busy

provider = IBMQ.load_account()

/opt/conda/lib/python3.7/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [2]:
# Let us write a function to create the circuit representing the CHSH game
def GHZ_circ(x, y, z):
    
    # x: bit received by Alice
    # y: bit received by Bob
    # z: bit received by Charlie
        
    circ = QuantumCircuit(3, 3)
    
    # First we set the |psi> state
    circ.h(0)
    circ.cx(0, 1)
    circ.h(1)
    circ.cx(1,2)
    circ.cx(0,2)
    circ.z(2)
    
    # Alice's qubit 
    if x == 1:
        circ.h(0)
            
    # Bob's qubit
    if y == 1:
        circ.h(1)
        
    # Charlie's qubit
    if z == 1:
        circ.h(2)
        
    circ.measure(range(3), range(3))
     
    return circ

In [3]:
# Now we need a function that will calculate the probability of winning the game
def win_prob(backend, shots = 1024):
    
    victories = 0
    
    games = [GHZ_circ(0, 0, 0), GHZ_circ(0, 1, 1), GHZ_circ(1, 0, 1), GHZ_circ(1, 1, 0)]
    job = execute(games, backend = backend, shots = shots)
    job_monitor(job)
    
    # We have four possible games
    # In the first case, they win if there are either none or two 1's
    counts = job.result().get_counts(games[0])
    if '000' in counts:
        victories += counts['000']
    if '011' in counts:
        victories += counts['011']
    if '101' in counts:
        victories += counts['101']
    if '110' in counts:
        victories += counts['110']
    # In the last three, they win if if there are either one or three 1's
    for game in games[1:4]:
        counts = job.result().get_counts(game)
        if '001' in counts:
            victories += counts['001']
        if '010' in counts:
            victories += counts['010']
        if '100' in counts:
            victories += counts['100']
        if '111' in counts:
            victories += counts['111']
    
    prob = victories/(4*shots)
    
    return prob

In [4]:
# First let us run the games in a simulator
backend = Aer.get_backend('qasm_simulator')
win_prob(backend)

Job Status: job has successfully run


1.0

In [5]:
# Now we run it in a real quantum device
backend = least_busy(provider.backends(operational = True, simulator = False, status_msg = 'active',
                            filters = lambda x: x.configuration().n_qubits >= 3))
print('Executing on:', backend)
win_prob(backend)

Executing on: ibmq_athens
Job Status: job has successfully run


0.917724609375